In [1]:
import pandas as pd 
import numpy as np
import json
import pandas_market_calendars as mcal

# Minutely Data

## Configuration

In [2]:
# List off functions for modifying the return

## Import IQFeed Stocks

1. Sort news by stock ticker
2. For each stock ticker do: 
3. Download prices for stock ticker
4. 

In [230]:
ticker = "FITB"

In [231]:
def get_next_available_candle(prices: pd.DataFrame, 
                              time: pd.Timestamp) -> pd.Series:
    entry_candle_idx = prices.index.get_indexer(target=[time], 
                                                method="bfill")
    entry_candle = prices.take(entry_candle_idx).iloc[0]
    return entry_candle

In [232]:
nyse_cal = mcal.get_calendar('NYSE')
def get_appropriate_closing_time(time: pd.Timestamp) -> pd.Timestamp:
    if (time.hour < 9) or ((time.hour == 9) and (time.minute < 30)):
        return pd.Timestamp(year=time.year, month=time.month, day=time.day, hour=16, minute=0, tz=time.tz)
    else:
        valid_days = [x.date() for x in nyse_cal.valid_days(start_date=time.date(), end_date=time.date() + pd.DateOffset(days=10))]
        i = 1
        while True:
            new_time = time + pd.DateOffset(days=i)
            if new_time.date() in valid_days:
                return pd.Timestamp(year=new_time.year, month=new_time.month, day=new_time.day, hour=16, minute=0, tz=time.tz)
            if i == 7:
                return ValueError()
            i += 1

## Import and Preprocess Stock Prices

In [265]:
prices: pd.DataFrame = pd.read_parquet(f"D:/IQFeedData/{ticker}_1min.parquet", columns=["time", "close", "volume"])
import pytz
eastern = pytz.timezone('US/Eastern')
prices.loc[:, "time"] = prices.loc[:, "time"].dt.tz_localize(None)
prices.loc[:, "time"] = prices.loc[:, "time"].dt.tz_localize(eastern)
prices.drop_duplicates(keep="first", inplace=True)
prices.dropna(inplace=True)

# Deals with duplicate rows which occurr when not all the digits for volume are 
# correctly entered, but only the first 1-3. So keep the largest.
prices = prices.sort_values(["time", "volume"], ascending=[True, False])
prices = prices.drop_duplicates(subset=["time"], keep="first")

prices.set_index("time", inplace=True)
prices.sort_index(ascending=True, inplace=True)
assert prices.index.is_unique
prices.head(3)

,close,volume
time,,
2010-01-04 08:03:00-05:00,9.75,7799.0
2010-01-04 08:06:00-05:00,9.75,67884.0
2010-01-04 08:13:00-05:00,9.84,300.0


## Import and Preprocess News 

In [233]:
df = pd.read_parquet("data/unraw2_bzg/data-5.parquet")

In [234]:
# Direct assignment doesnt work here, if df.time 
# already has a tz then it doesnt change on assignment 
tmp = df.time.dt.tz_convert(eastern)
df.loc[:, "time"] = 0
df.loc[:, "time"] = tmp

In [235]:
ts = df.time.dt.ceil("min").iloc[5]
print(ts)
new_ts = get_appropriate_closing_time(ts)
print(new_ts)

2011-04-08 07:00:00-04:00
2011-04-08 16:00:00-04:00


In [236]:
# TODO: This can be *improved* by saying that if we are very close to completing the minute e.g. :55, 
# then we dont take the next candle (T+1), but the candle after the next(T+2).
df.loc[:, "entry_time"] = df.loc[:, "time"].dt.ceil("min")
df.loc[:, "nn_exit_time"] = df.loc[:, "time"].apply(get_appropriate_closing_time)

In [237]:
ticker_news = df[df.stocks == ticker]

In [238]:
news_event = ticker_news.iloc[-1]

In [249]:
df.columns

Index(['time', 'stocks', 'author', 'title', 'channels', 'body', 'entry_time',
       'nn_exit_time'],
      dtype='object')

In [262]:
prices.index.is_monotonic_decreasing

False

## Make Input-Output and Merge

In [266]:
merged = pd.merge_asof(ticker_news, prices, left_on="entry_time", right_on="time", direction="forward")

In [267]:
merged = pd.merge(merged, prices, left_on="nn_exit_time", right_on="time", suffixes=("_entry", "_exit"))

In [272]:
merged["r"] = merged["close_exit"] / merged["close_entry"] - 1

In [273]:
merged

,time,stocks,author,title,channels,body,entry_time,nn_exit_time,close_entry,volume_entry,close_exit,volume_exit,r
0,2011-04-14 10:40:05-04:00,FITB,Benzinga,Shares of FITB Down 3.2% Since Downtrend Call ...,"[""Press Releases""]",(via COMTEX News Network)-- SmarTrend identif...,2011-04-14 10:41:00-04:00,2011-04-15 16:00:00-04:00,13.5550,10589.0,13.790,142594.0,0.017337
1,2011-04-21 10:38:05-04:00,FITB,Benzinga,Fifth Third Bancorp Has Returned 8.1% Since Sm...,"[""Press Releases""]",(via COMTEX News Network)-- SmarTrend identif...,2011-04-21 10:39:00-04:00,2011-04-25 16:00:00-04:00,12.9301,16405.0,12.920,357938.0,-0.000781
2,2011-04-21 11:36:05-04:00,FITB,Benzinga,Watch for Shares of Fifth Third Bancorp (FITB)...,"[""Press Releases""]",(via COMTEX News Network)-- SmarTrend has det...,2011-04-21 11:37:00-04:00,2011-04-25 16:00:00-04:00,13.1000,7973.0,12.920,357938.0,-0.013740
3,2011-04-25 12:14:04-04:00,FITB,Benzinga,"Shares of Fifth Third Bancorp Under Pressure, ...","[""Press Releases""]",(via COMTEX News Network)-- Fifth Third Banco...,2011-04-25 12:15:00-04:00,2011-04-26 16:00:00-04:00,12.9601,3900.0,13.050,174257.0,0.006937
4,2011-04-29 12:06:06-04:00,FITB,Benzinga,Downtrend Call Realized As Fifth Third Bancorp...,"[""Press Releases""]",(via COMTEX News Network)-- SmarTrend identif...,2011-04-29 12:07:00-04:00,2011-05-02 16:00:00-04:00,13.3100,14804.0,13.070,246634.0,-0.018032
5,2011-05-02 13:52:07-04:00,FITB,Benzinga,Fifth Third Bancorp (FITB) Crosses Pivot Point...,"[""Press Releases""]",(via COMTEX News Network)-- SmarTrend has det...,2011-05-02 13:53:00-04:00,2011-05-03 16:00:00-04:00,13.0800,5875.0,13.400,248086.0,0.024465
6,2011-05-03 16:16:08-04:00,FITB,Benzinga,Fifth Third Processing Solutions Expands Suite...,"[""Press Releases""]",## Deploying Encryption Solution for Secure Te...,2011-05-03 16:17:00-04:00,2011-05-04 16:00:00-04:00,13.4100,400.0,13.150,373011.0,-0.019389
7,2011-05-12 13:42:06-04:00,FITB,Benzinga,Fifth Third Bancorp Has Returned 9.0% Since Sm...,"[""Press Releases""]",(via COMTEX News Network)-- SmarTrend identif...,2011-05-12 13:43:00-04:00,2011-05-13 16:00:00-04:00,12.7000,119823.0,12.490,331491.0,-0.016535
8,2011-05-13 12:20:06-04:00,FITB,Benzinga,"Fifth Third Bancorp: Making a Move, Down 1.2% ...","[""Press Releases""]",(via COMTEX News Network)-- Fifth Third Banco...,2011-05-13 12:21:00-04:00,2011-05-16 16:00:00-04:00,12.5300,25414.0,12.470,126287.0,-0.004789
9,2011-05-13 13:08:35-04:00,FITB,Benzinga,Fifth Third Bancorp (FITB) Crosses Pivot Point...,"[""Press Releases""]",(via COMTEX News Network)-- SmarTrend has det...,2011-05-13 13:09:00-04:00,2011-05-16 16:00:00-04:00,12.5010,34100.0,12.470,126287.0,-0.002480


# Daily Data

## Import Stocks

In [ ]:
stocks = pd.read_pickle("data/stocks.pkl")

In [ ]:
stocks.head(3)

In [ ]:
stock_tickers = set(stocks.index.get_level_values("ID").unique())
len(stock_tickers)
#test

## Import Stories

In [19]:
#stories = pd.read_pickle("data/stories.pkl")
stories = pd.read_parquet("data/raw_bzg/story_df_raw_2019.parquet")

In [ ]:
stories_tickers = set(stories.stocks.unique())

## Parse stocks

In [ ]:
assert stories.stocks.dtype == stocks.index.dtypes[1]

In [ ]:
def add_targets(df):
    required_columns = ["Close", "High", "Low", "Open"]
    # df.loc[:, "IntradayReturn"] = df["Close"]/df["Open"] - 1
    df.loc[:, "CloseToCloseReturn"] = df["Close"] / df.shift(1)["Close"] - 1
    # df.loc[:, "NextDayReturn"] = df.shift(-1)["Close"] / df.shift(-1)["Open"] - 1
    # df.loc[:, "CloseToNextOpen"] = df.shift(-1)["Open"] / df["Close"] - 1
    return df

In [ ]:
stocks.index.dtypes

In [ ]:
stocks.loc[:, ["IntradayReturn", "NextDayReturn"]] = np.nan
stocks = stocks.swaplevel(0, 1).sort_index(ascending=[True, True])

In [ ]:
stocks = stocks.groupby("ID", as_index=False).apply(add_targets)
stocks.index = stocks.index.droplevel(None)

## Parse Stories

In [ ]:
from pandas.tseries.offsets import BDay

In [ ]:
# PARAMETER 
typ = "CloseToCloseReturn"

In [ ]:
def get_appropriate_date(timestamp, typ):
    if typ == "CloseToCloseReturn":
        # TODO: Some noise here due to closing auction?
        if timestamp.hour < 16: return timestamp.date()
        if timestamp.hour >= 16: return timestamp.date() +  BDay(1)

In [ ]:
# test
get_appropriate_date(stories.NewsTimestamp.iloc[4], typ="CloseToCloseReturn")

### Date assignment

In [ ]:
# If we use Intraday return then news should only be between 9:40 am and 4pm (us trading hours).
# If we use close-to-close return then news for this days CTC should be between yesterday 4pm and today 4pm. 
stories.loc[:, "Date"] = stories.NewsTimestamp.apply(lambda x: get_appropriate_date(x, typ))
stories = stories.astype({"Date":'datetime64[ns]'})

## Merging

In [ ]:
stories.rename(columns=dict(stocks="ID"), inplace=True)

In [ ]:
stories.columns

In [ ]:
stocks.columns

In [ ]:
dataset = stories[["Date", "NewsTimestamp", "ID", "body"]].\
    merge(stocks[
        [
            # "IntradayReturn", 
            # "NextDayReturn", 
            "CloseToCloseReturn"]
         ], on=["Date", "ID"], how="inner")

In [ ]:
dataset[dataset.isna().sum(axis=1) > 0]

In [ ]:
dataset = dataset.dropna()

	Date	NewsTimestamp	ID	body	CloseToCloseReturn

In [ ]:
dataset.to_pickle("data/dataset.pkl")

## Create train-test-split 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
dataset = pd.read_pickle("data/dataset.pkl")
assert dataset.index.is_unique

In [ ]:
test_size = 0.2
seed = 420
### Train-test split -> Auslagern
train_idx, test_idx = train_test_split(dataset.index, test_size=0.2, random_state=seed)

In [ ]:
test_idx

## Filter training set for general stock market events

In [ ]:
# Select S&P
SnP = stocks.query("ID == 'A0AET0'")

In [ ]:
SnP = add_targets(SnP)

In [ ]:
alpha = 0.1 # Percentage observations classified as too extreme to be used in the training set  

In [ ]:
target_col = "CloseToCloseReturn"

In [ ]:
lower_quantile = SnP.loc[:, target_col].quantile(alpha/2)

In [ ]:
upper_quantile = SnP.loc[:, target_col].quantile(1-alpha/2)

In [ ]:
print(f"Upper Quantile: {upper_quantile:.4f}. Lower Quantile: {lower_quantile:.4f}")

In [ ]:
mask = (SnP.loc[:, target_col] >= lower_quantile) & (SnP.loc[:, target_col] <= upper_quantile)

# Only select dates where SnP behaved calmly
allowed_dates = SnP.loc[mask, :].index.get_level_values("Date")

In [ ]:
# Now trim  training set
train_dat = dataset.loc[train_idx, :]
adj_train_idx = train_dat.loc[train_dat.Date.isin(allowed_dates)].index

In [ ]:
train_idx

In [ ]:
adj_train_idx

## Save train and test indices

In [ ]:
with open('data/dataset_train_test_idx.pkl', 'wb') as f:
    pickle.dump((adj_train_idx, test_idx), f)